In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("groq_api_key")

llm = ChatGroq(groq_api_key= groq_api_key, model= "Llama3-8b-8192")


In [ ]:
os.environ["HUGGINGFACE_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings= HuggingFaceEmbeddings(model= "all-MiniLM-L6-v2")


In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain



In [ ]:
import bs4
loader = WebBaseLoader(
    web_path="https://en.wikipedia.org/wiki/Async/await",
    bs_kwargs=dict(
        parse_only= bs4.SoupStrainer(
            class_= ("post-content", "post-title", "post-header")
        )
    )
)
docs = loader.load()
docs

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap= 50)
text_chunks = splitter.split_documents(docs)

vector_store = Chroma.from_documents(documents=text_chunks, embedding=embeddings)
retriever =vector_store.as_retriever()
retriever


In [ ]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    "you are an assistant for question answering tasks"
    "use the following pieces of retrieved context to answers"
    "the question, if you don't know the answer, say that you"
    "don\'t know, use three sentences maximum"
    "and keep the answer concise"
    "/n/n"
    "{context}"
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),

    ]
)


In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, contextualize_q_prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
response = rag_chain.invoke({"input": "What is async?"})
response

In [ ]:
response["answer"]

In [ ]:
history_aware_retriever  = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)


In [ ]:
question_answer_chain= create_stuff_documents_chain(llm, qa_prompt)
rag_chain= create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history =[]
question = "what is self reflection"
response_1 = rag_chain.invoke({"input": {question}, "chat_history": {chat_history}})

chat_history.extend(
    HumanMessage(content=question),
    AIMessage(content= response_1["answer"])

)

In [ ]:
question_2 = "tell me more about it?"
response_2 = rag_chain.invoke({"input": question_2, "chat_history": {chat_history}})
response_2["answer"]


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store["session_id"] = ChatMessageHistory
    return store["session_id"]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key= "input",
    history_messages_key= "chat_history",
    output_messages_key= "answer"
)


In [ ]:
conversational_rag_chain.invoke({"input": "what's task decomposition?"},
                                config={"configurable": {"session_id": "abs123" }},

                                )["answer"]

In [ ]:
conversational_rag_chain.invoke({"input": "what're common ways of doing it?"},
                                config={"configurable": {"session_id": "abs123" }},

                                )["answer"]